<h1><center> UFU - Federal University of Uberlândia</center></h1>

<h2><center>Undergraduate Program in Civil Engineering</center></h2>

<h3><center>SCIENTIFIC RESEARCH PROJECT</center><br>
TITLE: USING XGBOOST MODELS FOR DAILY RAINFALL PREDICTION 
<br>  
<br>  
STUDENT: Pedro Augusto Toledo Rios</h3>

<p>This notebook is part of a Scientific Research Project in the field of Computer Science/Data Analysis.</p>


# Classification

## Imports and Initial Configurations

In [ ]:
# Data Analysis and Preprocessing
import pandas as pd
import numpy as np
import random as rnd
import seaborn as sn

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Machine Learning Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
import itertools
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import ConfusionMatrixDisplay
import xgboost as xgb
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt


# Exploratory Data Analysis

In [ ]:
# Data Processing
missing_values = ['n/a', 'na', '*****', '*', '*******', ' -', '******', '5..84', '3..66', '3.3.21', 
                  '1..41', '********', '3.7.94', '354.59*', '564..79', '5.04.24', '21:36', '**********', 
                  '***', '*********', '03:18', '00:00', '03:48', '08:42', '03:06', '09:06', '01:30', 
                  '07:48', '09:12', '10:18', '01:24', '#VALUE!', '926,4923,8', '27/07/1902**21:36:00', 
                  '-', '926.4923.8']

# Loading the dataset and handling missing values
weather_data = pd.read_csv('C:/Users/auped/Desktop/IC CORREÇÕES/python 05-11/pesquisa/dadosclima_PERIODOCHUVOSO.csv', 
                           header=None, sep=';', na_values=missing_values)

# Defining column names
weather_data.columns = ['Max Temperature (°C)', 'Min Temperature (°C)', 'Avg Temperature (°C)', 
                        'Wind Speed (km/h)', 'Solar Radiation (cal/cm²/h)', 'Pressure (mb)', 
                        'Relative Humidity (%)', 'Rainfall (mm)', 'Month', 'Year', 'Rainy/Dry']

# Cleaning and transforming the "Year" column
weather_data['Year'] = weather_data['Year'].str.replace(',,', '')
weather_data['Year'] = weather_data['Year'].astype(int)

# Filtering pressure values within the valid range
weather_data = weather_data[(weather_data['Pressure (mb)'] >= 870) & (weather_data['Pressure (mb)'] <= 1100)]

# Storing relevant variables
combine = [weather_data]
rainfall = weather_data['Rainfall (mm)']

# Displaying the first five rows of the processed dataset
weather_data.head()

# Dropping missing values
weather_data.dropna()

# Displaying the number of missing values in each column
print(weather_data.isnull().sum())

### Creation of a new class to determine whether it rained on a given day

In [ ]:
#weather_data['Did it Rain?'] = rainfall

# Loop to populate the new column with 0 (no rain) and 1 (rain)
for weather_data in combine:    
    weather_data.loc[weather_data['Rainfall (mm)'] == 0, 'Did it Rain?'] = 0
    weather_data.loc[weather_data['Rainfall (mm)'] > 0, 'Did it Rain?'] = 1

# Displaying the first five rows of the table
weather_data.head()


In [ ]:
weather_data.describe()

In [ ]:
# Handling Missing Values

weather_data['Min Temperature (°C)'].fillna(method='ffill', inplace=True)   # ffill => fills with the last known temperature before the missing value
weather_data['Avg Temperature (°C)'].fillna(method='ffill', inplace=True)   # ffill => fills with the last known temperature before the missing value
weather_data['Wind Speed (km/h)'].fillna(method='ffill', inplace=True)      # ffill => fills with the last known wind speed before the missing value
weather_data['Solar Radiation (cal/cm²/h)'].fillna(method='ffill', inplace=True) # ffill => fills with the last known radiation before the missing value
weather_data['Relative Humidity (%)'].fillna(method='ffill', inplace=True)  # ffill => fills with the last known humidity rate before the missing value
weather_data['Rainfall (mm)'].fillna(method='ffill', inplace=True)          # ffill => fills with the last known rainfall amount before the missing value
weather_data['Pressure (mb)'].fillna(method='ffill', inplace=True)          # ffill => fills with the last known pressure before the missing value
weather_data['Did it Rain?'].fillna(method='ffill', inplace=True)           # ffill => fills with the last known value before the missing value
weather_data['Max Temperature (°C)'].fillna(method='ffill', inplace=True)   # ffill => fills with the last known temperature before the missing value

# Checking for missing values after the imputation process
print('\nMissing Values in the dataframe after processing:\n', weather_data.isnull().sum(), sep="")


In [ ]:
weather_data.describe()

### Code snippets to filter the DataFrame for a specific period of interest.


In [ ]:
# 70% for training / 30% for testing

start_year = 1980
end_year = 2020
training_start_year = 1980
training_end_year = 2008
testing_start_year = 2009
testing_end_year = 2019

test_data = weather_data.copy()


In [ ]:
test_data.drop(columns=['Rainfall (mm)'], axis=1, inplace=True)
# test_data.drop(columns=['Relative Humidity (%)'], axis=1, inplace=True)


# Machine Learning Models


## Creating Training and Testing DataFrames


In [ ]:
# Creating Training and Testing DataFrames
df_train = test_data[test_data['Year'] >= training_start_year]
df_train = test_data[test_data['Year'] <= training_end_year]
df_test = test_data[test_data['Year'] >= testing_start_year]
df_test = test_data[test_data['Year'] <= testing_end_year]

x_train = df_train.iloc[:, 0:7]
y_train = df_train[['Did it Rain?']]

x = df_test.iloc[:, 0:7]
y = df_test[['Did it Rain?']]

x.head()


## Model - XGBoost to Determine Whether It Rained or Not


In [ ]:
# Creating the XGBoost classifier
xgboost_classifier = xgb.XGBClassifier()

# Training the XGBoost classifier
xgboost_model = xgboost_classifier.fit(x, y)

# Testing

y_pred4 = xgboost_model.predict(x_train)


In [ ]:
# Evaluation Scores

print('Precision: %.3f' % precision_score(y_train, y_pred4))
print('Accuracy: %.3f' % accuracy_score(y_train, y_pred4))
print('Recall: %.3f' % recall_score(y_train, y_pred4))


In [ ]:
# Compute the confusion matrix
cm = confusion_matrix(y_train, y_pred4)

# Define class labels (replace with your own if necessary)
class_names = ['Negative Class', 'Positive Class']

# Plot the confusion matrix using Seaborn
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2)  # Adjust font size

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)

plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()
